In [1]:
# Import stuff

import re
from BCBio import GFF
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
import pysam

## Functions

In [2]:
def extract_genes():
    '''
    Function for extracting genes corresponding to BUSCO hits.
    Returns a SeqRecord object with one feature per BUSCO hit.
    '''
    # TODO: File path based on run name, take as input to the function
    file_tsv = open("../run_4_gen/full_table_4.tsv", 'r')

    # Extract BUSCO IDs, start and end from table of hits into SeqRecord object, each BUSCO as a SeqFeature
    busco_record = SeqRecord(seq='', id='hits')
    for line in file_tsv.readlines():
        hit = (re.search(r'(\S*)\sComplete\s(\S*)\s(\S*)\s(\S*)\s\S*', line))
        if hit:
            busco_record.features.append(SeqFeature(FeatureLocation(int(hit.group(3)), int(hit.group(4))), id=hit.group(1), type='gene', qualifiers={'contig': hit.group(2)}))

    file_tsv.close()

    # Match the BUSCOs to augustus predicted genes in gff file
    gff_records = []
    correct_genes = SeqRecord(seq='', id='correct_genes')
    limit_infos = dict(
            gff_type = ["gene"]) # Only want genes

    for busco in busco_record.features:
        filename = busco.id # gff filenames are {busco_id}.out.1
        i = 1
        while True:
            try:
                file_gff = open("../data/intermediate/run_4/augustus_output/predicted_genes/"+filename+".out."+str(i))
                for record in GFF.parse(file_gff, limit_info=limit_infos):
                    gff_records.append(record)
                i += 1
            except: # read all files for that BUSCO
                break

    # Find augustus predicted genes from the gff that match BUSCOs
    for rec in gff_records:
        for hit in busco_record.features:
            for feature in rec.features:
                if hit.location.start-1 == feature.location.start and hit.location.end == feature.location.end: # For some reason start has 1 nt diff...
                    #feature.qualifiers = {'contig': rec.id}
                    feature.id = rec.id # save as qualifier or id?
                    correct_genes.features.append(feature)
                    break

    file_gff.close()
    return correct_genes

In [3]:
def extract_mapped():
    '''
    Function used to extract only reads mapped to a reference.
    Really slow, only used to minimize the reads dataset
    '''
    pairedreads = pysam.AlignmentFile("allpaired2.bam", "wb", template=samfile)
    mapped_reads =[]

    for read in samfile.fetch():
        # Probably stupidly long if-statement
        if read.is_read1 and not read.is_secondary and not read.is_unmapped and not read.mate_is_unmapped and read not in mapped_reads:
            mapped_reads.append(read)
            pairedreads.write(read)
            print(read)
            print(samfile.mate(read))
            pairedreads.write(samfile.mate(read))

    return mapped_reads

In [4]:
def infer_region(genes):
    '''
    Function inferring lib-type of a specific region
    '''
        # Counters for the different lib-types   
    libs = {
        'fr_first': 0,
        'fr_second': 0,
        'rf_first': 0,
        'rf_second': 0,
        'ff_first': 0,
        'ff_second': 0,
        'undecided': 0
    }
    
    for gene in genes.features:
        contig = gene.id
        start = int(gene.location.start)
        stop = int(gene.location.end)
        strand = gene.strand
        reads = []
        # Get reads mapped to a specific contig and in a sequence range
        # Slow, only take 1000 reads?
        for read in samfile.fetch(contig, start, stop):
            if not read.mate_is_unmapped and read.is_read1:
                reads.append([read, samfile.mate(read)])

        
        if True: # strand == 1: # Does strandedness change things?
            # Check lib-type of reads 
            for read in reads:
                first = read[0]
                second = read[1]
                try:
                    lib = ''
                    if not first.is_reverse:
                        lib += 'f'
                    else: 
                        lib += 'r'
                    if not second.is_reverse:
                        lib += 'f'
                    else: 
                        lib += 'r'
                    # TODO: Make sure firststrand and secondstrand are correctly assigned
                    if first.reference_start > second.reference_start:
                        lib = lib[::-1]
                        lib += '_first'
                    elif first.reference_start < second.reference_start:
                        lib += '_second'
                    else:
                        lib = 'undecided'
                    libs[lib] += 1
                except: libs['undecided'] +=1 # Some reads missing start or end-values
        """
        elif strand == -1: # Gene on minus strand
            for read in reads:
                first = read[0]
                second = read[1]
                try:
                    lib = ''
                    if not first.is_reverse:
                        lib += 'f'
                    else: 
                        lib += 'r'
                    if not second.is_reverse:
                        lib += 'f'
                    else: 
                        lib += 'r'
                    # TODO: Make sure firststrand and secondstrand are correctly assigned
                    if first.reference_start > second.reference_start:
                        lib = lib[::-1]
                        lib += '_first'
                    elif first.reference_start < second.reference_start:
                        lib += '_second'
                    else:
                        lib = 'undecided'
                    libs[lib] += 1
                except: libs['undecided'] +=1 # Some reads missing start or end-values
        """
    
    return libs

In [47]:
def write_result(lib_dict):
    output = open('../data/output/result.txt', 'w+')
    output.write("Results of library inferring: \nLibrary type \t Reads \t Percent \n")
    
    total_reads = 0
    for i in lib_dict:
        total_reads += lib_dict[i]
    
    for i in lib_dict:
        percent = '{:.1%}'.format(lib_dict[i]/total_reads)
        output.write("%s \t %d \t %s\n" % (i, lib_dict[i], percent))

## Running

In [6]:
# bam-file with mapped reads, needs to be sorted and indexed
samfile = pysam.AlignmentFile("../data/intermediate/4_sorted.bam", "rb")

In [7]:
test = extract_genes()

In [8]:
print(test.features[0].id)

4


In [9]:
result = infer_region(test)

In [10]:
print(result)

{'fr_first': 4407, 'fr_second': 4066, 'rf_first': 22, 'rf_second': 18, 'ff_first': 2, 'ff_second': 5, 'undecided': 1}


In [48]:
write_result(result)